# Ma Soumission au Wild Tech Challenge




## 1. Import des packages nécessaires

In [3]:
import mysql.connector
import requests
from progressbar import ProgressBar
from pandas import DataFrame

## 2. Connexion à la Base de données et récupération de la liste d'addresses  

In [4]:
connection_params = {
    'host': "localhost",
    'user': "root",
    'password': "root",
    'database': "dataengineer",
}

# Request address datas
request = "SELECT * FROM address  "

with mysql.connector.connect(**connection_params) as db:
    with db.cursor() as c:
        c.execute(request)
        address_list = c.fetchall()
        print(len(address_list))


562


In [13]:
# Print 10 first addresses

for address in address_list[:10]:
    print(", ".join(address[1:4]))

318 CHE DE ROUMAGOUA, LA CIOTAT, 13600
19 RUE DES DAMES, SAINTE SAVINE, 10300
22 RUE AMIRAL GUEPRATTE, LE CONQUET, 29217
6 BD DES ETINES, LE COTEAU, 42120
35 AV DU 159EME RIA, BRIANCON, 5100
36 RUE D EN HAUT, BRUNEMONT, 59151
1625 RTE DE CHAMBERY, SAINT-ISMIER, 38330
3 RUE MONTESQUIEU, ASNIERES-SUR-SEINE, 92600
35 RUE PORTE ST JEAN, ORLEANS, 45000
67 BD COMMANDANT CHARCOT, NEUILLY-SUR-SEINE, 92200


## 3. Récupération des coordonnées de chaque adresse via API

Ici l'API que l'on devait normalement utiliser (Nominatim) produit un grand nombre d'erreurs (i.e. elle n'arrive pas à retrouver beaucoup d'addresses). J'ai donc utilisé une autre API par la suite mais commençons d'abord par Nominatim.

### 3.1 Avec l'API Nominatim

In [7]:
api_endpoint_1 = "https://nominatim.openstreetmap.org/search"

pbar = ProgressBar()
errors = []
for address in pbar(address_list):

    address_query = ' '.join(list(address)[1:4])
    address_query = address_query.lower()
    address_id = address[0]
    params = {'q': address_query, 'format': 'json'}
    response = requests.get(url=api_endpoint_1, params=params)

    # Check if an address is found by the API, if not add the address to the errors list
    if len(eval(response.text)) < 1:
        errors.append(address)

print(len(errors))  # 100 errors with this API

100


La liste "errors" contient la liste des adresses qui n'ont pas été retrouvées par l'API, il y en a donc 100 ici. 

In [12]:
# Print 5 first missing addresses

for error in errors[:5]:
    print(", ".join(error[1:4]))

35 AV DU 159EME RIA, BRIANCON, 5100
35 RUE PORTE ST JEAN, ORLEANS, 45000
117 RUE DES CHARMETTES, LYON 6EME, 69006
5 PL MARCEL PAGNOL, SAINT-NAZAIRE, 44600
2 RTE DU MOULIN, LES HAYS, 39120


### 3.2 Avec une autre API

In [9]:
api_endpoint_2 = "https://api-adresse.data.gouv.fr/search/"
pbar = ProgressBar()
errors_2 = []
coordinates = []

for address in pbar(address_list):

    # Set up the queries to give to the API
    address_query = ' '.join(list(address)[1:4])
    address_query = address_query.lower()
    address_id = address[0]

    # Set up the API call
    params = {'q': address_query, 'format': 'json'}
    response = requests.get(url=api_endpoint_2, params=params)

    # Get API call result
    result = response.json()['features']

    # If results not found
    if len(result) < 1:
        errors_2.append(address_query)
        coordinates.append([address_id, 'NULL', 'NULL'])

    # If results found get the coordinates
    else:
        lat = result[0]['geometry']['coordinates'][0]
        long = result[0]['geometry']['coordinates'][1]
        coordinates.append([address_id, lat, long])

print(len(errors_2))  # Plus qu'une seule erreur avec cette API


1


In [10]:
errors_2

['28 lot mont vernon i saint martin 97150']

Ici il n'y a plus qu'une seule adresse non retrouvée. J'aurais pu insérer manuellement les coordonées pour cette adresse mais je vais plutôt laisser les valeurs NULL car j'imagine que c'est ce que je ferais en production.

## 4. Insertion des coordonnées dans la base de donnée

In [ ]:

# Create the columns
query_1 = "ALTER TABLE address ADD latitude DECIMAL(8,5);"
query_2 = "ALTER TABLE address ADD longitude DECIMAL(8,5);"

with mysql.connector.connect(**connection_params) as db:
    with db.cursor() as c:
        c.execute(query_1)
        c.execute(query_2)


# Fill the columns
with mysql.connector.connect(**connection_params) as db:
    with db.cursor() as c:
        for coord in coordinates:

            # If the address wasn't found by the API, set NULL value
            if coord[1] == 'NULL':
                query = f"UPDATE address SET latitude = NULL, longitude = NULL WHERE address_id = {int(coord[0])}"
                print(query)
                c.execute(query)

            else:
                query = f"UPDATE address SET latitude = {round(coord[1],5)}, longitude = {round(coord[2],5)}" \
                        f" WHERE address_id = {int(coord[0])}"
                print(query)
                c.execute(query)
        db.commit()

### 5. Qui a fait le plus de location ? 

In [11]:
query = "SELECT c.first_name, c.last_name, a.address_id, a.address, a.latitude, \
         a.longitude,  COUNT(r.rental_id) as count_rent	\
         FROM customer AS c \
         JOIN address as a USING(address_id) \
         JOIN rental as r USING(customer_id) \
         GROUP BY c.customer_id \
         ORDER BY count_rent DESC \
         LIMIT 1"

with mysql.connector.connect(**connection_params) as db:
    with db.cursor(buffered=True) as c:
        print(query)
        c.execute(query)
        df = DataFrame(c.fetchall())
        df.columns = c.column_names

df.head()



SELECT c.first_name, c.last_name, a.address_id, a.address, a.latitude,          a.longitude,  COUNT(r.rental_id) as count_rent	         FROM customer AS c          JOIN address as a USING(address_id)          JOIN rental as r USING(customer_id)          GROUP BY c.customer_id          ORDER BY count_rent DESC          LIMIT 1


,first_name,last_name,address_id,address,latitude,longitude,count_rent
0,ELEANOR,HUNT,152,17 IMP DES JARDINS,5.96634,46.10690,46


La personne ayant fait le plus de location est Madame Eleanor Hunt avec 46 locations